In [1]:
import os

# Define the new working directory relative to the current working directory
# or use an absolute path
new_working_directory = os.path.join(os.getcwd(), '..', '..', '..') # Set to MLAgentBenhc
print("New working directory: ", new_working_directory)

# Change the current working directory
os.chdir(new_working_directory)

print("New Working Directory:", os.getcwd())

New working directory:  c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench\MLAgentBench_v2\agents\Voyager\..\..\..
New Working Directory: c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench


In [2]:
from MLAgentBench_v2.LLM import complete_text_openai
from MLAgentBench_v2.agents.agent import Agent
import numpy as np 
import json
import logging 
import os
import re
import subprocess
from pathlib import Path
import shutil
import time 
import types
import copy

No module named 'helm'
Could not load CRFM API key crfm_api_key.txt.
[Errno 2] No such file or directory: 'claude_api_key.txt'
Could not load anthropic API key claude_api_key.txt.


In [3]:
# Instantiate an environment

from types import SimpleNamespace
from MLAgentBench_v2.environment import Environment

args = SimpleNamespace(
    task='home-data-for-ml-course',
    task_type='kaggle',
    log_dir='logs/house-price-testing_simple-assistant-agent_gpt35_v1',
    work_dir='workspace',
    max_steps=50,
    max_time=18000,
    device=0,
    python='/home/user/micromamba/envs/autogpt/bin/python',
    interactive=False,
    resume=None,
    resume_step=0,
    agent_type='SimpleAssistantAgent',
    llm_name='gpt-3.5-turbo-1106',
    fast_llm_name='gpt-3.5-turbo-1106',
    edit_script_llm_name='gpt-3.5-turbo-1106',
    edit_script_llm_max_tokens=4000,
    agent_max_steps=50,
    actions_remove_from_prompt=[],
    actions_add_to_prompt=[],
    no_retrieval=False,
    valid_format_entires=None,
    max_steps_in_context=3,
    max_observation_steps_in_context=3,
    max_retries=4,
    langchain_agent='zero-shot-react-description'
)

env = Environment(args)

Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/house-price-testing_simple-assistant-agent_gpt35_v1', work_dir='workspace', max_steps=50, max_time=18000, device=0, python='/home/user/micromamba/envs/autogpt/bin/python', interactive=False, resume=None, resume_step=0, agent_type='SimpleAssistantAgent', llm_name='gpt-3.5-turbo-1106', fast_llm_name='gpt-3.5-turbo-1106', edit_script_llm_name='gpt-3.5-turbo-1106', edit_script_llm_max_tokens=4000, agent_max_steps=50, actions_remove_from_prompt=[], actions_add_to_prompt=[], no_retrieval=False, valid_format_entires=None, max_steps_in_context=3, max_observation_steps_in_context=3, max_retries=4, langchain_agent='zero-shot-react-description')
Preparing task home-data-for-ml-course , of type:  kaggle


In [5]:
goal = env.research_problem
print("Goal: ", goal)

Goal:  Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Evaluation
Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. You want a train and validation MAE of lower than 15,000 and there should be a submission.csv containing predictions for test.csv ready to submit.

Metric
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting ex

In [6]:
root_file_directory = "workspace"

In [7]:
exploration_progress = "None"

In [10]:
class CurriculumAgent(Agent):
    def __init__(self, env, completed_tasks=[], failed_tasks=[]):
        super().__init__(env)
        self.completed_tasks = completed_tasks
        self.failed_tasks = failed_tasks

        # The crux is a Q&A process
        # Problem with this approach is you still have to deal with searching multiple times, and continuing to search or not. Approach: Or maybe if you search and you don't have the answer, that's a bad thing to search and you need to go more specific / ask a different question!
        self.system_prompt_automatic_curriculum = f'''You are a helpful assistant that tells me the next immediate task to do. My ultimate goal is to discover as many useful pieces of information as possible to better achieve the research goal, answer as many questions as possible to get the best answer, and become the best researcher in the world in solving this research goal.

        Research Goal: {self.research_problem}

        I will give you the following information:
        Skills: these are skills that I can take action with.
        Files: these are my current files that I have in my working directory.
        Most recent files, action, result, and answer states (oldest to newest): Answer states are the a report of the best answer I have so far to achieving the research goal, and the files, action, and result are the files, action, and result I took at that point in time to produce the answer state.    
        Completed tasks so far: ...
        Failed tasks that are too hard: ...

        1) You should act as a mentor and guide me to the next task based on my current learning progress.
        2) Please be very specific about what information or actions I need to take and what expected results I need to achieve.
        3) The next task should follow a clear format, such as "Write [file]", "Reflect on why I'm seeing [error]", "Read [file]", "Brainstorm if ideas from [topic 1] be applied to [topic 2]", "Analyze what are the similarities between [topic 1] for success and [topic 2]" , "Reflect on what's significant about this paper: [paper]?", "Reflect on what's currently missing or preventing me from achieving [goal] better", etc. It should be a single task to collect useful information on. Do not propose multiple tasks at the same time. Do not mention anything else. 
        4) The next task should not be too hard since the internet and I may not contain the full answer in a single article or have learned enough information to complete it yet. 
        5) The next task should be novel and interesting based on my current learning progress. I should look for rare and potentially useful pieces of information, upgrade my current answer using better information, and discover new things. I should not be doing the same thing over and over again.
        6) I may sometimes need to repeat some tasks or variations of the task if I need to collect more information to answer more difficult tasks. Only repeat tasks if necessary. 
        7) I want to explore the world and discover new things. I don’t want to stay with my current answer for too long. 
        8) Tasks that require information beyond another reader's ability to theoretically verify and reason if completed or correct should be avoided. For instance, "what else is there on the website?" and "what images and tables are on the website" are not ideal since they require visual confirmation from the screen. All the testing, coding, and asking other people questions should be avoided. Do not propose a task  with these keywords. You should only respond in the format as described below:
        
        RESPONSE FORMAT: 
        ```json
        {{ 
            "reasoning": "<based on the information I listed above, do reasoning about what the next task should be.>",
            "task": "<the next task.>"
        }}
        ```
        
        Here’s an example response: 
        ```json
        {{ 
            "reasoning": "We know that we have a sword and we know there's fire, and fire lights things on fire. Therefore, we could try to make a firesword.",
            "task": "Try to make a firesword and record what happens."
        }}
        ```

        Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc. This is important.
'''

        # TODO: This is optional, might be useful, but to focus on a system prompt of asking questions and answering questions.
        # System 2: this is a more scoped down version where we have the focus be on only answering questions -- reading and analyzing information & asking questions. No action items. 
        # The current above system 1 is better for self-driving labs type of work where there are going to be more tasks.

    def get_exploration_progress(self):
        # TODO: this should contain inventory of where we're at now and what files we have / memory stream
        return f'''Completed tasks: {self.completed_tasks}, Failed tasks: {self.failed_tasks}'''

    def propose_next_task(self):
        '''
        This function decomposes a goal into tasks
        '''        
        user_prompt = f'''Files: {self.files}
Skills: {self.available_actions.keys()}
Most recent files, action, result, and answer states (oldest to newest):
{self.formatted_answer_states}      
Completed tasks so far: {self.completed_tasks}
Failed tasks that are too hard: {self.failed_tasks}
'''
        
        print("System prompt for generating curriculum: \n", self.system_prompt_automatic_curriculum, "\n User prompt: ", user_prompt)
        next_task_response = complete_text_openai(user_prompt, system_prompt=self.system_prompt_automatic_curriculum, json_required=True)
        print("Response: ", next_task_response)
        next_task = json.loads(next_task_response)["task"]
        return next_task

    def get_completed_tasks():
        pass

    def get_failed_tasks():
        pass

    def add_completed_task(self, task):
        # TODO: probably we should record the entire answer state of files, action, output, and answer state? Or just action and output?
        self.completed_tasks.append(task)

    def add_failed_task(self, task):
        self.failed_tasks.append(task)

curriculum_agent = CurriculumAgent(env)

In [11]:
curriculum_agent.propose_next_task()

System prompt for generating curriculum: 
 You are a helpful assistant that tells me the next immediate task to do. My ultimate goal is to discover as many useful pieces of information as possible to better achieve the research goal, answer as many questions as possible to get the best answer, and become the best researcher in the world in solving this research goal.

        Research Goal: Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Evaluation
Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the 

"Read 'train.csv' and 'test.csv' to understand the structure and contents of the dataset."

In [7]:
class SkillManager():
    # Additional skills to be added: await bot.findsProblem, await bot.findsInterestingInformation, await bot.findSolution or something like that

    def __init__(self):
        self.READ_MAX_CHAR_SIZE = 2500
        self.functions = [
            {
                "name": "read_file",
                "description": "Get the text from the file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {
                            "type": "string",
                            "description": "the complete file name and extension, e.g. abc.txt"
                        }
                    },
                    "required": ["file_name"],            
                }
            }, 
            {
                "name": "write_file",
                "description": "Write text to a file",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {
                            "type": "string",
                            "description": "the complete file name and extension, e.g. abc.txt"
                        },
                        "text": {
                            "type": "string",
                            "description": "the text to write to the file"
                        }
                    },
                    "required": ["file_name", "text"],            
                }
            }, 
            {
                "name": "web_search",
                "description": "Given a prompt, I'll return back my web search information about it",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Tell me what to search about"
                        }
                    },
                    "required": ["query"],            
                }
            },
            {
                "name": "think",
                "description": "Given a prompt, I'll return back my thoughts on it.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Tell me what to think about"
                        }
                    },
                    "required": ["query"],            
                }
            },
            # {
            #     "name": "num_children",
            #     "description": "This paper talks about how many girls and boys are in this family",
            #     "parameters": {
            #         "type": "object",
            #         "properties": {},
            #         "required": [],
            #     },
            # },
            
            # {
            #     "name": "content_in_data_txt",
            #     "description": "I know the information in data.txt",
            #     "parameters": {
            #         "type": "object",
            #         "properties": {},
            #         "required": [],
            #     },
            # }
            # {'name': 'columns_and_descriptions', 'description': "The question was what are the columns and their descriptions in the training set and the test set. The answer is that the columns and their descriptions can be found in the 'data_description.txt' file using the 'read_file' function. The columns and their descriptions in both the training set and the test set include information about the type of dwelling, zoning classification, lot size, road and alley access, property shape and flatness, utilities available, and lot configuration.", 'parameters': {'type': 'object', 'properties': {}, 'required': []}}
        ]

        self.available_functions = {
            "read_file": self.read_file,
            "write_file": self.write_file,
            "web_search": self.web_search,
            "think": self.think,
        }

        # Currently not much utility, but if you needed to trace back to a function or fact it got wrong, this can help.
        # TODO: honestly, not really sure what to do with this. I was thinking seeing the entire message history would be a good way to trace back, but we already have the original message prompt
        self.function_history = [
            "given",
            "given",
            "given",
            "given",
        ]

        self.files = [
            ("train.csv", "the training set"),
            ("test.csv", "the test set"),
            ("data_description.txt", "full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here"),
            ("sample_submission.csv", "a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms"),
            ("data_fields.txt", "a brief version of what you'll find in the data description file."),
        ]

    def retrieve_skills(self, task, execution_feedback):
        # For sake of simplicity, use recency for now (later relevancy and importance can be added)
        func_name_description_list = [str((func["name"], func["description"])) + '\n' for func in self.functions]
        return func_name_description_list
    
    def retrieve_info_blocks(self, task, execution_feedback):
        # retrieving file names and description
        return [name + " - " + description for name, description in self.files]

    # Helper function: be able to create a function with a dynamic name and return value
    # def create_skill_function(self, function_name, return_value):
    #     def dynamic_method(self):
    #         return return_value
    #     # Bind the function to the instance as a method
    #     bound_method = types.MethodType(dynamic_method, self)
    #     setattr(self, function_name, bound_method)
    #     # Add the method to available functions
    #     self.available_functions[function_name] = bound_method

    # Core function: adding a new skill requires an original task, a validated answer, and a message history
    def add_skill(self, task, validated_answer, methods_prompt):
        # TODO: wait until the action agent generates a function because maybe you only need to write a description of the input function instead of task and validated answer.
        # create_function_description_system_prompt = f'''You are a helpful assistant that writes a description of the given '''

        print("Adding skills! ", task, validated_answer, methods_prompt)

        create_skill_system_prompt = f'''You are a helpful assistant. Your goal is to write a short file name and a short description of the question and answer. 
        
        You will receive this information:
        Original task or question: ...
        Answer: ...

        Do not use any of these file names: {[name for name, _ in self.files]}

        Your output should be in the following format if function requires arguments:
        ```json
        {{
            "name": "<file_name>",
            "description": "<insert question and answer>"
        }}
        ```

        Good example output:
        ```json
        {{
            "name": "num_dogs_in_bens_family",
            "description": "The question was how many dogs are in the family. Ben said that he has 2 dogs in his family."
        }}
        ```

        Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc. This is important.
        '''

        create_function_description_prompt = f'''
        Original task or question: {task}
        Answer: {validated_answer}
        '''
        res, messages = complete_text_openai(prompt=create_function_description_prompt, system_prompt=create_skill_system_prompt, verbose=True)
        res

        try:
            # Load the function description
            file_name_description = json.loads(res['content'])
            print("file_name_description: ", file_name_description)

            # Create the function as a method of skill_manager
            self.write_file(file_name_description['name'], f"Question: {task}\nAnswer: {validated_answer}\nReasoning and Methods: {methods_prompt}")

            # Add function to function description list
            self.files.append((file_name_description['name'], file_name_description['description']))

            print("COMPLETE!")
            
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return

        return

    # Below are given functions or dummy functions
    def read_file(self, file_name):
        '''Get the text from the file'''
        file_path = f'{root_file_directory}/{file_name}'
        with open(file_path, 'r') as file:
            text = file.read()[:self.READ_MAX_CHAR_SIZE]
        return text
    
    def write_file(self, file_name, text):
        '''Write the given text to the file.'''
        file_path = f'{root_file_directory}/{file_name}.txt'
        with open(file_path, 'w') as file:  # 'w' is the mode for writing to a file
            file.write(text)  # Write the entire text to the file
        return text

    def web_search(self, query):
        # Temporarily just return search info from the user
        return input(f"Search for {query} on the web: ")
        
    def think(self, query):
        return complete_text_openai(query + " Think step by step.", verbose=True)[0]['content']
    
    # Demo of how to add plain just fact / info skills that gets added as just natural language?
    def content_in_data_txt(self):
        return '''The data.txt file contains the following information: File descriptions
train.csv - the training set
test.csv - the test set
data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
data_fields.txt - a brief version of what you'll find in the data desc'ription file.'''

    def num_children(self):
        return '''There are 2 boys, and 8 girls'''
    
    # MANUALLY ADDED FUNCTIONS FOR TESTING
    def columns_and_descriptions():
        return '''The question was what are the columns and their descriptions in the training set and the test set. The answer is that the columns and their descriptions can be found in the 'data_description.txt' file using the 'read_file' function. The columns and their descriptions in both the training set and the test set include information about the type of dwelling, zoning classification, lot size, road and alley access, property shape and flatness, utilities available, and lot configuration.'''
    
skill_manager = SkillManager()
print("Available functions: \n", skill_manager.available_functions, "\n\nFunctions", skill_manager.functions, "\n\nAvailable files: \n", skill_manager.retrieve_info_blocks("", ""))
# Unit test to ensure that the skill manager's basic skills work
skill_manager.write_file("hello_world.txt", "hello_world")
assert(os.path.isfile(f'{root_file_directory}/hello_world.txt'))
read_file_result = skill_manager.read_file("hello_world.txt")
assert(read_file_result == "hello_world")
web_search_result = skill_manager.web_search("Do penguins fly?")
assert(len(web_search_result) > 0 and type(web_search_result) == str)
# think_result = skill_manager.think("Do penguins fly?") # Commenting out just for now since it costs to test
# assert(len(think_result) > 0 and type(think_result) == str)

Available functions: 
 {'read_file': <bound method SkillManager.read_file of <__main__.SkillManager object at 0x00000279272E9850>>, 'write_file': <bound method SkillManager.write_file of <__main__.SkillManager object at 0x00000279272E9850>>, 'web_search': <bound method SkillManager.web_search of <__main__.SkillManager object at 0x00000279272E9850>>, 'think': <bound method SkillManager.think of <__main__.SkillManager object at 0x00000279272E9850>>} 

Functions [{'name': 'read_file', 'description': 'Get the text from the file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}}, 'required': ['file_name']}}, {'name': 'write_file', 'description': 'Write text to a file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}, 'text': {'type': 'string', 'description': 'the text to write to the file'}},

FileNotFoundError: [Errno 2] No such file or directory: 'agent_files/hello_world.txt.txt'

In [ ]:
class ActionAgent():
    # TODO: does there need to be an action agent generating the steps? Or does there need to be a separate execution agent running the prompt? Or can the action agent be the execution agent?
    # TODO: can a critic agent really check if the output is correct? Or can they only check if that aligns with expectation? Otherwise, the critic will have to check the line of content values to make sure the reasoning is sound, which is still doable, but the extent that another critic can check is limited. I guess it's just to make sure the reasoning is sound.

    def __init__(self):
        pass

    def generate_function_callable_prompt(self, task,
                methods_prompt,
                execution_feedback,
                execution_errors,
                critique,
                skills,
                info_blocks,
                skill_manager):
        generate_methods_system_prompt = '''You are a helpful assistant and a first-rate problem solver. Given a task or question, your goal is to list out the steps to solve that task or question given your skills and reasoning. Ultimately, your output should be able to be followed by a human limited by the skills and knowledge given, and another human should be able to check that human's output to see if it's correct and reasonable. Note that the functions asked for may sometimes already be called and the information from the function that you need is already in the prompt, so read carefully. Note that you DO NOT have access to run any code, you can only read, think, and write about the existing skills and knowledge.
        
        You will be given this information:
        Task or question: ...
        Skills: You only have these skills to help you write the steps to achieve this task or answer this question.
        Knowledge: You only have these information blocks as files that you can read to get more information about the details of the information block's answer and the reasoning and methods used to arrive at the answer.
        Current state steps: ...
        Current state output after executing steps: ...
        Execution errors: ...
        Critique: ...
        
        You should then respond to me with
        Explain (if applicable): Are there any steps missing in your plan? Why do the current state steps not lead to the answer? What do the current state output, execution errors, and critique imply?
        Steps: 
        1) Write how to answer the question and complete the task step by step. 
        2) Reuse the above useful knowledge as much as possible. The question answering check is ultimately based on how reasonable your steps are and if the final answer after following your steps is a complete answer to the question.
        3) Your steps will be reused to trace back the reasoning behind the answer.
        4) The final answer will be reused for building more complex information blocks. Therefore, you should make the final answer reusable.
        5) You should not make strong assumptions about knowledge and information, and therefore you should always check whether you have the required information before using them in reasoning. If not, you should first collect the required information and reuse the above useful knowledge.
        5) Do not propose steps that the above skills cannot accomplish.
        '''

        user_prompt = f'''Task or question: {task}
        Skills: {skills}
        Knowledge: {info_blocks}
        Current state solution: {methods_prompt}
        Current state output after executing steps: {execution_feedback}
        Execution errors: {execution_errors}
        Critique: {critique}'''

        agent_methods_feedback, agent_methods_errors = self.function_call(generate_methods_system_prompt, user_prompt, skill_manager)
        # TODO: parse out just the steps to save
        # agent_methods_feedback_json = json.loads(agent_methods_feedback['content'])

        # TODO: Add error handling for if the agent_methods_feedback is empty and agent_methods_errors exists
        print("agent_methods_feedback", agent_methods_feedback, "agent_methods_errors", agent_methods_errors)

        # steps_idx = agent_methods_feedback['content'].find("Steps: ")
        # steps = agent_methods_feedback['content'][steps_idx + len("Steps: "):].strip()

        return f'Task or question: {task} \nInstructions: ' + agent_methods_feedback['content']
    
    def function_call(self, system_prompt, methods_prompt, skill_manager):
        print()
        try:
            response_message, messages = complete_text_openai(system_prompt=system_prompt, prompt=methods_prompt, tools=skill_manager.functions, available_functions=skill_manager.available_functions,verbose=True)
        except Exception as e:
            return "", e
        return response_message, None
    
action_agent = ActionAgent()
methods_prompt = None
execution_feedback = None
execution_errors = None
critique = None
success = False

In [ ]:
class ExecutionAgent():
    # Ultimately the answer will go into Description or be "returned" with the description being condensed into a tldr and the methods_prompt will be added to history

    # TODO: Perhaps the execution agent can get info about the files too, but maybe that's the action agent's responsibility.

    def function_call(self, methods_prompt, skills, info_blocks, skill_manager):
        system_prompt = f'''You are a helpful assistant. Your goal is to execute the given instructions and output the complete answer to the question. If the instructions don't seem reasonable or you cannot get to the complete answer, then you should give feedback on why you couldn't do it and what you tried. 

        You will be given this information:
        Skills: You only have these skills to help you execute the instructions.
        Knowledge: You only have these information blocks as files that you can read to get more information about the details of the information block's answer and the reasoning and methods used to arrive at the answer.
        Instructions: ...
'''
        execute_prompt = f'''Skills: {skills}
        Knowledge: {info_blocks}
        Instructions: {methods_prompt}'''

        try:
            response_message, messages = complete_text_openai(system_prompt=system_prompt, prompt=execute_prompt, tools=skill_manager.functions, available_functions=skill_manager.available_functions, verbose=True)
            conclusion = response_message

            # # Assuming response_message['content'] is a string that contains 'Conclusion:'
            # match = re.search(r'Conclusion:(.*)', response_message['content'], re.DOTALL)
            # if match:
            #     conclusion = match.group(1).strip()
            # else:
            #     conclusion = response_message['content']  # or some default value, in case "Conclusion:" isn't found
        except Exception as e:
            return "", e
        return conclusion, None
execution_agent = ExecutionAgent()

In [ ]:
class CriticAgent():
    # TODO: Perhaps include the skills to the critic so the critic knows the facts to check if this makes sense or not

    def check_task_success(self, task, methods_prompt, execution_feedback, skills, info_blocks, skill_manager):
        system_prompt = '''You are an assistant that assesses my progress of research and provides useful guidance. 
        
        You are required to evaluate if I have provided a correct and direct answer to the question. Providing more information and exceeding the task requirements is also considered a success while failing to meet them or not actually answering the question requires you to provide critique to help me improve. Note that me providing steps is not enough, my final answer must be provided and directly answer the question.

        I will give you the following information:
        Skills: My skills. 
        Knowledge: My information blocks as files.
        Task or question: The question I need to answer.
        Answer: My current answer.
        Approach: My reasoning of how I got to my answer.

        You should only respond in JSON format as described below:
        {
            "biggest_risk", "biggest risk for why the answer might not answer the question",
            "reasoning": "reasoning for if the answer successfully answers the question or not",
            "success": boolean,
            "critique": "critique",
        }
        Ensure the response can be parsed by Python "json.loads", e.g.: no trailing commas, no single quotes, etc.

        Here are some examples:
        INPUT:
        Task or question: What is the distribution of the sale prices in the dataset?
        Answer: To determine the distribution of the sale prices in the dataset, we can follow these steps:\n\n1. Read the dataset file "train.csv" using the `read_file` function.\n2. Extract the column containing the sale prices from the dataset.\n3. Calculate the frequency of each unique sale price in the dataset.\n4. Sort the unique sale prices in ascending order.\n5. Create a histogram or bar chart to visualize the distribution of the sale prices.\n6. Optionally, you can also calculate summary statistics such as mean, median, and standard deviation of the sale prices.\n\nLet\'s start by reading the dataset file "train.csv".
        Approach: Task or question: What is the distribution of the sale prices in the dataset? \nInstructions: To determine the distribution of the sale prices in the dataset, you can follow these steps:\n\n1. Read the dataset file "train.csv" using the `read_file` function.\n2. Extract the column containing the sale prices from the dataset.\n3. Calculate the frequency of each unique sale price in the dataset.\n4. Sort the unique sale prices in ascending order.\n5. Create a histogram or bar chart to visualize the distribution of the sale prices.\n6. Optionally, you can also calculate summary statistics such as mean, median, and standard deviation of the sale prices.\n\nPlease note that the specific implementation details may vary depending on the programming language and libraries you are using.

        RESPONSE:
        {
            "biggest_risk", "There is no actual distribution in the answer.",
            "reasoning": "The reasoning to get to the answer makes sense, but there's no direct answer for what the actual distribution of the sale price is.",
            "success": False,
            "critique": "The answer only tells us how to get the distribution is, but does not tell us what the actual distribution. Please tell us what the actual distribution is.",
        }
        '''

        user_prompt = f'''Skills: {skills}
        Knowledge: {info_blocks}
        Task or question: {task}
        Answer: {execution_feedback}
        Approach: {methods_prompt}'''

        response_message, messages = complete_text_openai(system_prompt=system_prompt, prompt=user_prompt, verbose=True)

        response_json = json.loads(response_message['content'])
        success = response_json['success']
        reasoning = response_json['reasoning']
        critique = response_json['critique']

        # lines = response_message['content'].split("\n")
        # for line in lines:
        #     # Strip whitespace for accurate matching
        #     line = line.strip()
            
        #     # Check if the line starts with the known titles and parse accordingly
        #     if line.startswith('Reasoning:'):
        #         critique = line[len('Reasoning:'):].strip()
        #     elif line.startswith('Task is reasonably answered:'):
        #         success = line[len('Task is reasonably answered:'):].strip().lower()

        print("System prompt: ", system_prompt, "\n\nUser prompt: ", user_prompt, "\n\nResponse: ", response_message['content'])
        return success, critique
critic_agent = CriticAgent()

In [ ]:
# Instantiate only once! 
skill_manager = SkillManager()

In [ ]:
# START HERE FOR NEXT ITERATIONS: Save the current iteration's state after this cycle in case we need to revert
skill_manager_copy = copy.deepcopy(skill_manager)
curriculum_agent_copy = copy.deepcopy(curriculum_agent)

In [ ]:
# REVERT BY STARTING HERE, uncomment the below and run
skill_manager = copy.deepcopy(skill_manager_copy)
curriculum_agent = copy.deepcopy(curriculum_agent_copy)

In [ ]:
print("curriculum_agent.completed_tasks", curriculum_agent.completed_tasks, "curriculum_agent.failed_tasks", curriculum_agent.failed_tasks)

curriculum_agent.completed_tasks [] curriculum_agent.failed_tasks []


In [ ]:
next_question = curriculum_agent.propose_next_question(skills=skill_manager.retrieve_skills(task="", execution_feedback=""), files=skill_manager.retrieve_info_blocks(task="", execution_feedback=""), exploration_progress=[])
print("next_question", next_question)

System prompt for generating curriculum: 
 You are a helpful assistant that asks questions to help me decide the next immediate question to answer on the computer. My ultimate goal is to discover as many useful pieces of information as possible, answer as many questions as possible, and become the best researcher in the world.

        Goal: 
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

Metric
Submissions are ev

NameError: name 'openai_client' is not defined

In [ ]:
print("\nCurrent files\n", skill_manager.retrieve_info_blocks(task="", execution_feedback=None))


Current files
 ['train.csv - the training set', 'test.csv - the test set', 'data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here', 'sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms', "data_fields.txt - a brief version of what you'll find in the data description file.", 'methods_to_analyze_relationships - The question was about common methods used to analyze the relationships between variables and a target variable in a regression problem like predicting house prices. The answer provides a list of methods including scatter plots, correlation analysis, feature importance, and multivariate analysis. It also suggests documenting the findings and insights from the analysis for future reference and reporting.']


In [ ]:
print(skill_manager.functions)
print(skill_manager.available_functions)

[{'name': 'read_file', 'description': 'Get the text from the file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}}, 'required': ['file_name']}}, {'name': 'write_file', 'description': 'Write text to a file', 'parameters': {'type': 'object', 'properties': {'file_name': {'type': 'string', 'description': 'the complete file name and extension, e.g. abc.txt'}, 'text': {'type': 'string', 'description': 'the text to write to the file'}}, 'required': ['file_name', 'text']}}, {'name': 'web_search', 'description': "Given a prompt, I'll return back my web search information about it", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Tell me what to search about'}}, 'required': ['query']}}, {'name': 'think', 'description': "Given a prompt, I'll return back my thoughts on it.", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'de

In [ ]:
# Attempting to build 1 cycle with all agents
# next_question = "Search up how many penguins there are in Kevin Huang's house. We don't need an exact or true number."
# next_question = "What is the total number of penguins and children that we know about from our functions?" # hardcoded for now
num_rounds = 2
num_tasks = 1
print("next_question", next_question)

next_question What is the distribution of the target variable (SalePrice) in the training set? Are there any potential outliers?


In [ ]:
for task_idx in range(num_tasks):
    methods_prompt = None
    execution_feedback = None
    execution_errors = None
    critique = None
    success = False
    for i in range (num_rounds):
        print(f"\nRound {i} Task: \n", next_question)
        
        skills = skill_manager.retrieve_skills(next_question, execution_feedback)
        info_blocks = skill_manager.retrieve_info_blocks(next_question, execution_feedback)
        print("\nSkill manager output:\nSkills: ", skills, "\nInfo blocks: ", info_blocks)

        print("\nStarting action agent")
        action_agent = ActionAgent()
        methods_prompt = action_agent.generate_function_callable_prompt(next_question, methods_prompt, execution_feedback, execution_errors, critique, skills, info_blocks, skill_manager)
        print("\nAction agent output:\n", methods_prompt)

        print("\nStarting execution agent")
        execution_agent = ExecutionAgent()
        execution_feedback, execution_errors = execution_agent.function_call(methods_prompt, skills, info_blocks, skill_manager)
        print("\nExecution agent output: ", execution_feedback, "\nExecution errors: ", execution_errors)

        if execution_errors:
            continue

        print("\nStarting critic agent")
        critic_agent = CriticAgent()
        success, critique = critic_agent.check_task_success(next_question, methods_prompt, execution_feedback, skills, info_blocks, skill_manager)
        print("Critic agent output", "\nSuccess: ", success, "\nCritique: ", critique)

        if success:
            break
    if success:
        print("\nBefore adding info block\n", skill_manager.retrieve_info_blocks(next_question, execution_feedback=None))
        skill_manager.add_skill(next_question, execution_feedback, methods_prompt)
        print("\nAfter adding info block\n", skill_manager.retrieve_info_blocks(next_question, execution_feedback=None))
        curriculum_agent.add_completed_task("Question: " + next_question + " \nAnswer: " + execution_feedback)
    else:
        curriculum_agent.add_failed_task("Question: " + next_question + " \nBest answer: " + execution_feedback + " \nCritique for why it failed: " + critique)


Round 0 Task: 
 What is the distribution of the target variable (SalePrice) in the training set? Are there any potential outliers?

Skill manager output:
Skills:  ["('read_file', 'Get the text from the file')\n", "('write_file', 'Write text to a file')\n", '(\'web_search\', "Given a prompt, I\'ll return back my web search information about it")\n', '(\'think\', "Given a prompt, I\'ll return back my thoughts on it.")\n'] 
Info blocks:  ['train.csv - the training set', 'test.csv - the test set', 'data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here', 'sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms', "data_fields.txt - a brief version of what you'll find in the data description file.", 'methods_to_analyze_relationships - The question was about common methods used to analyze the relationships between va

Completion info:  {
  "id": "chatcmpl-8HnjCCmWGhiAed2h2U70uouUF1r5L",
  "object": "chat.completion",
  "created": 1699253422,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Steps:\n1) Read the \"train.csv\" file to get the training set data.\n2) Extract the \"SalePrice\" column from the training set data.\n3) Calculate the distribution of the \"SalePrice\" column.\n4) Check for potential outliers in the \"SalePrice\" column.\n5) Write the distribution and information about potential outliers to a file for future reference."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 960,
    "completion_tokens": 78,
    "total_tokens": 1038
  }
}
Function calling response:  {
  "role": "assistant",
  "content": "Steps:\n1) Read the \"train.csv\" file to get the training set data.\n2) Extract the \"SalePrice\" column from the training set data.\n3) Calculate the distribut

TypeError: can only concatenate str (not "NoneType") to str